In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt
import math
import os
from itertools import product
from datetime import datetime
from dateutil.relativedelta import relativedelta
from quant_functions import anal_funcs, data_funcs

import yfinance as yf
## FRED 정보 수집
from fredapi import Fred

In [1]:
# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go

from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)

In [2]:
## 그리기 설정값
color_basic = "#3b4a6b" # 네이비 기본색
color_norm = "#f23557" # 진홍색 강조색
color_cont = "#22b2da" # 파란색 대비색
color_sub = "#ff6f3c" # 주황색 보조색
color_around = "#6E6E6E" # 회색 주변색
color_around2 = "#E0E0E0" # 연회색 주변색

figsize=(12, 8)

##### Function #####

##### Data Load #####

In [7]:
index = ['최저1인', '최저2인', '최저3인', '최저4인', '중위1인', '중위2인', '중위3인', '중위4인', '상위']
need_cash_ls = [134, 221, 283, 344, 224, 368, 471, 573, 1584]
need_cash_df = pd.DataFrame([index, need_cash_ls], index=["index", "need_cashflow_month"]).T
need_cash_df['need_cashflow_annual'] = need_cash_df['need_cashflow_month']*12
need_cash_df['dividend_rate'] = 0.0333
need_cash_df['tax_rate'] = 0.154
need_cash_df['need_kapital'] = (need_cash_df['need_cashflow_annual']/(need_cash_df['dividend_rate']*(1-need_cash_df['tax_rate']))).astype(int)

In [8]:
need_cash_df

,index,need_cashflow_month,need_cashflow_annual,dividend_rate,tax_rate,need_kapital
0,최저1인,134,1608,0.0333,0.154,57078
1,최저2인,221,2652,0.0333,0.154,94136
2,최저3인,283,3396,0.0333,0.154,120546
3,최저4인,344,4128,0.0333,0.154,146529
4,중위1인,224,2688,0.0333,0.154,95414
5,중위2인,368,4416,0.0333,0.154,156752
6,중위3인,471,5652,0.0333,0.154,200626
7,중위4인,573,6876,0.0333,0.154,244073
8,상위,1584,19008,0.0333,0.154,674717
